In [32]:
import torch
import numpy as np
import os
import pickle as pkl
from copy import copy
from typing import Dict, Union
import CardiacMesh
from CardiacMesh import Cardiac3DMesh

In [38]:
root_folder = f"{os.environ['HOME']}/01_repos/CardiacCOMA/data/cardio/meshes"
# root_folder = "/home/rodrigo/CISTIB/UKBB/data/meshes/FHM/Results"

Select (at most) $N$ subjects:

In [40]:
N = 1000
ids = os.listdir(root_folder)
ids = ids[:N]

# Mesh generator

Load 3D meshes at ED (timeframe `001`)

In [12]:
def get_3d_mesh(ids, root_folder):
    
    for id in ids:
        try:
          npy_file = f"{root_folder}/{id}/models/FHM_time001.npy"
          pc  = np.load(npy_file)
          yield id, pc
        except:
          continue
        

In [13]:
meshes_gen = get_3d_mesh(ids, root_folder)
meshes = {}

for id, mesh in meshes_gen:
    meshes[id] = mesh        

___

# Apply Procrustes transforms

In [31]:
QSLIM_OUTPUT_10PCT = "data/faces_and_downsampling_mtx_frac_0.1_full_heart.pkl"
PROCRUSTES_FILE = "data/procrustes_transforms_FHM_35k.pkl"
procrustes_transforms = pkl.load(open(PROCRUSTES_FILE, "rb"))

In [37]:
id = "1000215"
CardiacMesh.transform_mesh(meshes[id], **procrustes_transforms[id])

array([[ 40.93693583,  -2.34215486,   2.82250658],
       [ 25.54735976,  -5.88779846, -76.92301678],
       [ 46.28188747, -17.79776513,  -4.14224227],
       ...,
       [ 19.0399027 ,  37.13969888,  13.43696795],
       [ 13.21599531,  43.09471474,  13.61656523],
       [ 12.38542294,  42.61489267,  16.89599939]])

___

# Display transformed meshes

In [20]:
import ipywidgets as widgets
from ipywidgets import interact
import pyvista as pv
import random

In [28]:
# sphere = vedo.Sphere(res=params["mesh_resolution"]).to_trimesh()
# conn = sphere.faces # connectivity
# conn = np.c_[np.ones(conn.shape[0]) * 3, conn].astype(int)  # add column of 3, as required by PyVista

pv.set_plot_theme("document")

faces, _ = pkl.load(open(QSLIM_OUTPUT_10PCT, "rb")).values()
faces = np.c_[np.ones(faces.shape[0]) * 3, faces].astype(int)

color_palette = list(pv.colors.color_names.values())
random.shuffle(color_palette)


def f(ids, rotated, traslated):
                
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    
    for i, id in enumerate(ids):
      
        mesh = meshes[id]
        if rotated and not traslated:
            mesh = CardiacMesh.transform_mesh(mesh, rotation=procrustes_transforms[id]["rotation"])
        elif traslated and not rotated:
            mesh = CardiacMesh.transform_mesh(mesh, traslation=procrustes_transforms[id]["traslation"])
        elif traslated and rotated:
            mesh = CardiacMesh.transform_mesh(mesh, **procrustes_transforms[id])
        mesh = pv.PolyData(mesh, faces)
          
        pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[i], opacity=0.2)
    
    pl.show(interactive=True, interactive_update=True)
    
#interact(f, i=widgets.IntSlider(min=1,max=N))
interact(f, 
    ids=widgets.SelectMultiple(options=ids),
    rotated=widgets.ToggleButton(),
    traslated=widgets.ToggleButton()
);

interactive(children=(SelectMultiple(description='ids', options=('1004985', '1007167', '1007980', '1005995', '…